<a href="https://colab.research.google.com/github/aNOnWhyMooS/connectivity/blob/main/notebooks/interpol_eval_plot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Clone Repository and install requirements

In [ ]:
!git clone https://github.com/aNOnWhyMooS/connectivity

In [ ]:
%%capture
!pip install transformers datasets flax optax
!pip install GitPython tabulate gpytorch

In [ ]:
%cd /content/connectivity_gems/
!git pull origin main

In [ ]:
!python3 -m pip install -e src/
!bash get_paws.sh

## Evaluation

In [ ]:
%cd /content/connectivity_gems/evaluate/glue

/content/connectivity_gems/evaluate/glue


In [ ]:
!python3 eval_models.py --base_models_prefix connectivity/bert_ft_qqp- --dataset paws --split dev_and_test --models 0 1 2 3\
                       --write_file_prefix eval_qqp- --batch_size 128

## Interpolate

In [ ]:
%cd /content/connectivity_gems/interpolate

/content/connectivity_gems/interpolate


### 1-D

In [ ]:
!python3 interpolate_1d.py --base_models_prefix connectivity/bert_ft_qqp- --dataset qqp --split validation\
                          --save_file interpol.pkl --suffix_pairs 7,22 7,98 22,98 1,7 1,98 --batch_size 128

### 2-D

In [ ]:
!python3 interpolate_2d.py --base_models_prefix connectivity/feather_berts_ --anchor 99 --base1 44 --base2 87\
                          --dataset hans --split test --metric ECE 

## Plot

### Get Logs

In [ ]:
%cd /content/connectivity_gems/

/content/connectivity_gems


In [ ]:
!mkdir logs/
!python3 get_logs.py logs/
!rm  logs/*.lock

In [ ]:
!cd logs; git clone https://huggingface.co/connectivity/interpolation_logs/

Cloning into 'interpolation_logs'...
remote: Enumerating objects: 1971, done.
remote: Counting objects: 100% (1971/1971), done.
remote: Compressing objects: 100% (1669/1669), done.
remote: Total 1971 (delta 857), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (1971/1971), 11.43 MiB | 8.42 MiB/s, done.
Resolving deltas: 100% (857/857), done.


### 1-D interpolations

In [ ]:
%%bash
cd /content/connectivity_gems/plot/
sufs="";for i in {0..99}; do sufs="$sufs $i";done;
python3 peak_valley_plains.py --perf_metric lexical_overlap_onlyNonEntailing --interpol_datasets MNLI\
                              --interpol_log_dirs ../logs/interpolation_logs/mnli_interpol@36813steps/\
                              --eval_mods_prefix ../logs/hans_eval_bert_ --eval_mods_suffixes $sufs --remove_plains

### 2-D interpolations

In [ ]:
%%bash
cd /content/connectivity_gems/plot/
export BASE_DIR=../logs/interpolation_logs/interpol_2d/short_range
python3 same_z_scale_plot.py --surface_pkl_files $BASE_DIR/around_peaks/mnli_test/mnli_test_99_8_37_2_loss_surface.pkl\
                                                 $BASE_DIR/around_valleys/mnli_test/mnli_test_44_73_89_2_loss_surface.pkl\
                                                 $BASE_DIR/peak_and_2valleys/mnli_test/mnli_test_99_44_73_2_loss_surface.pkl\
                              --plot_title "" --names '(a.) generalized models' '(b.) heuristic models'\
                                                      '(c.) generalized and heuristic models' \
                              --point_names G0 G1 G2 H0 H1 H2 G0 H0 H1 --clip_x -0.5 1.5 --clip_y -1.0 1.20 --clip_z 0 0.65

### Heatmaps and Scatter Plots

In [ ]:
%%bash
cd /content/connectivity_gems/plot/
sufs="";for i in {0..99}; do sufs="$sufs $i";done;
python3 interpol_heatmap.py --order_by perf --eval_metric f1 \
                    --interpol_log_dir ../logs/interpolation_logs/qqp_interpol@34110steps/ \
                    --eval_mods_prefix ../logs/paws_eval@34110steps_bert_ft_qqp-\
                    --eval_mods_suffixes $sufs --emb_acc_corr --ticks accs

### Training Dynamics

In [ ]:
%%bash
cd /content/connectivity_gems/plot/
sufs="";for i in {0..99}; do sufs="$sufs $i";done;
export BASE_DIR=../logs/interpolation_logs/qqp_interpol@
python3 dynamics.py --eval_metric f1 --interpol_log_dirs ${BASE_DIR}15000steps/ ${BASE_DIR}25000steps ${BASE_DIR}34110steps \
                    --eval_mods_prefixes ../logs/paws_eval@34110steps_bert_ft_qqp- ../logs/paws_eval@34110steps_bert_ft_qqp-\
                    ../logs/paws_eval@34110steps_bert_ft_qqp- --eval_mods_suffixes $sufs